In [412]:
import numpy as np
class Cell:
    def __init__(self, initial_value = False):
        if not initial_value:
            self.domain = set([1,2,3,4,5,6,7,8,9])
        else:
            assert initial_value in range(1,10)
            self.domain = set([initial_value])
            
    def remove(self, item):
        if item in self.domain:
            self.domain.remove(item)
            return True
        else:
            return False
        
    def get_known_cell_value(self):
        assert len(self.domain) == 1
        for elem in self.domain:
            return elem
    
    def is_known(self):
        return len(self.domain) == 1

empty_cell = "."
class Sudoku:
    board = [["empty" for _ in range(9)] for _ in range(9)]
    known_indices = set([])
    unknown_indices = set([])
    def __init__(self, list_board):
        #assume '.' denodes empty cells, '4' would be a valued cell
        for row in range(9):
            for col in range(9):
                list_item = list_board[row][col]
                if list_item == empty_cell:
                    self.board[row][col] = Cell()
                    self.unknown_indices.add((row, col))
                else:
                    self.board[row][col] = Cell(int(list_item))
                    self.known_indices.add((row, col))
                    
    def get_cell(self, row, col):
        return self.board[row][col]
                    
    def print_board(self):
        for row in self.board:
            buff = []
            for cell in row:
                if cell.is_known():
                    buff.append(cell.get_known_cell_value())
                else:
                    buff.append(empty_cell)
            print(*buff)
    
    def return_board(self):
        board_list = []
        for row in self.board:
            buff = []
            for cell in row:
                if cell.is_known():
                    buff.append(str(cell.get_known_cell_value()))
                else:
                    buff.append(empty_cell)
            board_list.append(buff)
        return board_list
    
    def row_indices(self, row, col):
        return set([(row_i, col) for row_i in range(9) if row_i != row])
    
    def col_indices(self, row, col):
        return set([(row, col_i) for col_i in range(9) if col_i != col])
        
    def sub_block_indices(self, row, col):
        sub_row = row // 3
        sub_col = col // 3
        to_return = set([(3*sub_row + row_i, 3*sub_col + col_i) for row_i in range(3) for col_i in range(3)])
        to_return.remove((row, col))
        return to_return
    
    def get_constrained_indices(self, row, col):
        return self.row_indices(row, col).union(self.col_indices(row, col)).union(self.sub_block_indices(row, col))
    
    def iterate_consistency(self):
        removed_count = 0
        now_known_indices = set([])
        for known_index in self.known_indices:
            cell = self.get_cell(*known_index)
            cell_value = cell.get_known_cell_value()
            for constrained_index in self.unknown_indices.intersection(self.get_constrained_indices(*known_index)):
                constrained_cell = self.get_cell(*constrained_index)
                if constrained_cell.remove(cell_value) == True:
                    removed_count += 1
                    if constrained_cell.is_known():
                        self.unknown_indices.remove(constrained_index)
                        now_known_indices.add(constrained_index)
                        
        self.known_indices = self.known_indices.union(now_known_indices)
        self.unknown_indices.difference_update(now_known_indices)
        return removed_count
    
    def iterate_arc_consistency(self):
        '''ex: if unknown_cell is the only cell in its row that can be a 2, unknown cell must be a 2'''
        now_known_indices = set([])
        for unknown_index in self.unknown_indices:
            unknown_cell = self.get_cell(*unknown_index)
            
            row_domain = set([])
            for row_index in self.row_indices(*unknown_index):
                row_domain = row_domain.union(self.get_cell(*row_index).domain)
            if len(unknown_cell.domain.difference(row_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(row_domain)
                now_known_indices.add(unknown_index)
                continue
                
            col_domain = set([])
            for col_index in self.col_indices(*unknown_index):
                col_domain = col_domain.union(self.get_cell(*col_index).domain)
            if len(unknown_cell.domain.difference(col_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(col_domain)
                now_known_indices.add(unknown_index)
                continue
                
            sub_block_domain = set([])
            for sub_index in self.sub_block_indices(*unknown_index):
                sub_block_domain = sub_block_domain.union(self.get_cell(*sub_index).domain)
            if len(unknown_cell.domain.difference(sub_block_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(sub_block_domain)
                now_known_indices.add(unknown_index)
                continue
        
        self.known_indices = self.known_indices.union(now_known_indices)
        self.unknown_indices.difference_update(now_known_indices)

In [413]:
test_input = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]
test_input2 = [[".",".","9","7","4","8",".",".","."],["7",".",".",".",".",".",".",".","."],[".","2",".","1",".","9",".",".","."],[".",".","7",".",".",".","2","4","."],[".","6","4",".","1",".","5","9","."],[".","9","8",".",".",".","3",".","."],[".",".",".","8",".","3",".","2","."],[".",".",".",".",".",".",".",".","6"],[".",".",".","2","7","5","9",".","."]]
S = Sudoku(test_input)
print("Start")
S.print_board()

Start
5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9


In [414]:
import time
start = time.time()
while len(S.known_indices) < 81:
    print("known:", len(S.known_indices))
#     print(np.array(S.return_board()))
    S.iterate_consistency()
    S.iterate_arc_consistency()
print(time.time() - start)

known: 30
known: 48
known: 76
0.008753061294555664


In [374]:
test = set([1,2,3])
test.add(*[(1,2)])
print(test.difference((1,2)))
print(test)

{(1, 2), 3}
{(1, 2), 1, 2, 3}


In [375]:
test = set([])
test.union(set([4]))
print(test)












set()
